# GP Sample Paths

This Jupyter notebook includes demonstrations of various (real-valued) Gaussian Processes (GPs). In particular,

* Section 1 demonstrates how to simulate White Gaussian Noise (WGN) and how it sounds.
* Section 2 does the same things for Wiener Processes (Brownian Motion), which are non-stationary GPs.
* Section 3 provides two methods, RARP and DARP, for sampling band-limited, stationary, zero-mean GPs.
* Section 4 demonstrates how to simulate band-pass filtered (ans, as a special case, low-pass filtered) WGNs using the methods of the previous section and how they sound.
* Section 5 does the same things for bandlimited, baseband, stationary Colored Gaussian Noise. 

Note that "simulate a random process (RP)", "draw realizations (or sample paths) from an RP" and "sampling a RP" are all semantically the same.


To use this notebook, run all cells and hop from demo to demo.

**Notes:**
 - Requires Python 3.x; tested on Python 3.7.6
 - Requires `numpy`, `matplotlib`, `ipywidgets`, `IPython.display`.
 - To install `ipywidgets`, use
    - `conda install ipywidgets` or `conda install -c conda-forge ipywidgets`
 - or
    - `pip install ipywidgets`
    - `jupyter nbextension enable --py --sys-prefix widgetsnbextension`


Authored by Georgios C. Anagnostopoulos 
ver. 1.0 (April 2020)

In [47]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import Audio, display

import functools

In [48]:
# Some global settings for figure sizes
normalFigSize = (8, 6) # (width,height) in inches
largeFigSize = (12, 9)
xlargeFigSize = (18, 12)

## 1. White Gaussian Noise

_White Gaussian Noise_ (WGN) of intensity $\sigma^2 > 0$ is a zero-mean, stationary Gaussian Process (GP) with time-averaged auto-correlation function

\begin{align*}
    \bar{R}_X(\tau) \equiv R_X(t+\tau, t) = \sigma^2 \delta(\tau)
\end{align*}

and a power spectral density (PSD) of

\begin{align*}
    S_X(j \omega) = \sigma^2
\end{align*}

i.e., constant for all $\omega$. Its 'whiteness' characterization stems from the fact that its power is distributed evenly over all frequencies in a fashion similar to white (colorless) light; no frequency (color) dominates the others in terms of power.

Among all continuous-time Random Processes (RPs), it is probably the most straightforward to draw realizations from. Assume that $X$ is a WGN with constant PSD $\sigma^2$ and we want to draw a realization $x$ from it with samples $x_k \triangleq x(t_k)$ at times $\left\{ t_k \right\}_{k=0}^N$ for some $N \geq 1$. It suffices to draw $N$ i.i.d samples $x_k$ from $\mathcal{N}(0, \sigma^2)$ and then graph them. That's what we'll do next.

**Commnent:**

* Wide-Sense Stationary (WSS) and Strict-Sense Stationarity are one and the same for GPs. Hence, we will be plainly characterize a GP as "stationary" or "non-stationary".  
* As a reminder, for a WSS RP $X$, its time-averaged auto-correlation function $\bar{R}_X(\tau)$ coincides with their auto-correlation function $R_X(t + \tau, t)$, which is independent of $t$, like the case above. 

### 1.1 WGN Simulation

In [49]:
# Global variables & objects
sigmaSquared_max = 2.0

sigmaSquaredSlider = widgets.FloatSlider(
    value=0.5,
    min=0.0,
    max=sigmaSquared_max,
    step=0.1,
    description='$\sigma^2$',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [50]:
def plotSamplePaths_WGN(sigmaSquared = sigmaSquaredSlider, showGrid=True):
    
    t_min = 0.0 # sec
    t_max = 100.0 # sec
    Nt = 1000
    t = np.linspace(t_min, t_max, Nt)
    
    x = np.random.normal(0.0, np.sqrt(sigmaSquared), Nt)

    fig, ax = plt.subplots(1, 1, figsize=normalFigSize)
    ax.plot(t, x, lw=1.0, color='blue')
    ax.set_xlim(t_min, t_max)
    maxDev = 1.96 * np.sqrt(sigmaSquared_max) # corresponds to 95% prediction interval 
    ax.set_ylim(-maxDev, maxDev) 
    ax.set_xlabel('$t$ (sec)',fontsize=18)
    ax.set_ylabel('$x(t)$', fontsize=18)
    ax.grid(showGrid)
    
v = interactive(plotSamplePaths_WGN, {'manual': True})
display(v)

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='$\\sigma^2$', max=2.0, read…

**Instructions:**

* To get a different sample path each time, click the `Run Interact` button.

**Observations:**

* Increasing $\sigma^2$ amplifies the RP; i.e., it "inreases its volume".

### 1.2 WGN Audio Sample

Next, let's listen to it.

In [51]:
# Global variables
durationSeconds = 3.0 # sec

In [52]:
# Also uses sigmaSquaredSlider
def soundSamplePaths_WGN(sigmaSquared = sigmaSquaredSlider):    
      
    t_min = 0.0
    t_max = durationSeconds
    # Note to self: for some reason, signals that are sampled at less than 3KHz,
    # will not play in the widget.
    samplingRateHz = 44100 # Hz; audio CD quality sampling rate 
    samplingPeriodSec = 1.0 / samplingRateHz # sec
    t = np.arange(t_min, t_max, samplingPeriodSec, dtype=float) 

    x = np.random.normal(0.0, np.sqrt(sigmaSquared), len(t))
    
    fig, ax = plt.subplots(1, 1, figsize=normalFigSize)
    ax.plot(t, x, lw=1.0, color='blue')
    ax.set_xlim(t_min, t_max)
    #ax.set_ylim(-10.0, 10.0)
    ax.set_xlabel('$t$',fontsize=18)
    ax.set_ylabel('$X(t)$', fontsize=18)
    #ax.grid(showGrid)
    
    display(Audio(data=x, rate=samplingRateHz))
    return x

v = interactive(soundSamplePaths_WGN, {'manual': True})
display(v)

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='$\\sigma^2$', max=2.0, read…

**Instructions:**

* To get a different sample path and audio each time, click the `Run Interact` button.
* You can download the generated audio as a `WAV` file by clicking on the three vertical dots to the left of the audio widget and select `Download`. 

**Observations:**

* WGN sounds like "static" noise.
* Changing $\sigma^2$ should change the volume of the noise, but we do not observe this, since the signal is likely first normalized in terms of loudness before it is played back to us. 

## 2. Wiener Process

A _Wiener Process_ (WP) $B$ is a zero-mean, non-stationary GP with the following defining characteristics:

1. $B(0) = 0$ with probability 1
2. The WP's increments $B(t+\tau) - B(t)$ for $\tau > 0$ are $\mathcal{N}(0, \sigma^2 \tau)$-distributed for some parameter $\sigma^2>0$. When $\sigma^2=1$, the RP is called a _stamdard_ WP.
3. The WP has independent increments over non-oberlapping intervals. For example, if $t_1 < t_2 < t_3 < t_4$, then the RVs $B(t_2) - B(t_1)$ and $B(t_4) - B(t_3)$ are independent. Such increments are jointly normal.
4. $B$ has continuous paths with probability 1 (i.e., almost surely). 

A WP is also referred to as a _Brownian motion_ in honor of the botanist Robert Brown, who observed in 1827 the motion of pollen in water. Turns out that the motion of this pollen can be modelled well by a two-dimensional WP. It was Norbert Wiener (whom the process took its name after), who showed in 1923 that WPs have MS-continuous paths, but are nowhere MS-differentiable. Later on, in 1905, it was argued by none other than Einstein that this Brownian motion was the first physical evidence of the existence of atoms and molecules, which until then were by large only theoretical constructs.

WPs have too many interesting properties to be covered in brief, one of the most elegant being its self-similarity property. Here, we are going only to consider a WP's (time-averaged) auto-correlation, the resulting PSD and its simulation.

Without going into details, we have that, for $t, \tau \geq 0$,

\begin{align*}
    & \mathbb{E} \! \left\{ B(t) \right\} = 0 & t \geq 0
    \\
    & R_B(t+\tau, t) = \sigma^2 \min \left\{ t + \tau, t \right\} & t+\tau, t \geq 0
\end{align*}

i.e., $B$ is a zero-mean, non-stationary GP and $B(t)$ is $\mathcal{N}(0, \sigma^2 t)$-distributed. It can be shown that $\frac{d B(t)}{dt} = W(t)$, where $W$ is a WGN with constant spectral density $\sigma^2$. Given that the squared magnitude response of a differentiator is $|H(j \omega)|^2 = |j \omega|^2 = \omega^2$, one can argue that the PSD of $B$ can be taken as

\begin{align*}
    & S_W(j \omega) = |H(j \omega)|^2 S_B(j \omega)  \quad \Rightarrow \quad S_B(j \omega) = \frac{\sigma^2}{\omega^2}
\end{align*}

The aforementioned PSD implies that $B$'s average power is infinite ($P_B = +\infty$). The same result is obtained, when attempting to compute the time-averaged auto-correlation $\bar{R}_X$ at lag 0. It turns out, that the computation of $\bar{R}_X$ at other lags is also infinite.

The simulation of such a WP can be accomplished by the process' 1st and 2nd defining properties. We choose a time resolution $\Delta t > 0$ (the smaller, the more accurate the realization) and then compute the values of the process $B_k \triangleq B(t_k)$ at time $t_k = k \Delta t$ via the recursion

\begin{align*}
    & B_o = 0 & 
    \\
    & B_k = B_{k-1} + \sqrt{\sigma^2 \Delta t} Z_k & k = 1, 2, \ldots 
\end{align*}

where the $Z_k$'s are i.i.d. $\mathcal{N}(0,1)$ samples. From this algorithm, one can see the connection of WPs to so-called _random walks_ : after each sample is drawn, the process has an equal probability to increase or decrease.


### 2.1 WP Simulation

In [53]:
# Uses sigmaSquaredSlider
def plotSamplePaths_WP(sigmaSquared = sigmaSquaredSlider, showGrid=True):
    
    t_min = 0.0 # do not change
    t_max = 100.0
    Nt = 1000
    t = np.linspace(t_min, t_max, Nt)
    
    x = np.empty(Nt)
    multiplier = np.sqrt(sigmaSquared * (t[1] - t[0]))
    x[0] = 0
    for k in range(1, Nt):
        x[k] = x[k-1] + multiplier * np.random.normal(0.0, 1.0)
    
    fig, ax = plt.subplots(1, 1, figsize=normalFigSize)
    ax.plot(t, x, lw=1.0, color='blue')
    ax.set_xlim(t_min, t_max)
    maxDev = 1.96 * np.sqrt(sigmaSquared_max * t_max) # corresponds to 95% prediction interval of x(t_max)
    ax.set_ylim(-maxDev, maxDev) 
    ax.set_xlabel('$t$ (sec)',fontsize=18)
    ax.set_ylabel('$x(t)$', fontsize=18)
    ax.grid(showGrid)
    
v = interactive(plotSamplePaths_WP, {'manual': True})
display(v)

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='$\\sigma^2$', max=2.0, read…

**Instructions:**

* To get a different sample path each time, click the `Run Interact` button.

**Observation:**

* Increasing $\sigma^2$ allows the WP stray further away from 0 within the displayed time frame.


**Comment:**

* While sample paths of a WP may exhibit increasing or decreasing trends within an observational time window, its mean is zero for any time instance.

### 2.2 WP Audio Sample

Let's take a listen to a WP sample path.

In [54]:
# Uses sigmaSquaredSlider
def soundSamplePaths_WP(sigmaSquared = sigmaSquaredSlider):    
      
    t_min = 0.0
    t_max = durationSeconds
    # Note to self: for some reason, signals that are sampled at less than 3KHz,
    # will not play in the widget.
    samplingRateHz = 44100 # Hz; audio CD quality sampling rate 
    samplingPeriodSec = 1.0 / samplingRateHz # sec
    t = np.arange(t_min, t_max, samplingPeriodSec, dtype=float) 
    Nt = len(t)
    
    x = np.empty(Nt)
    multiplier = np.sqrt(sigmaSquared * samplingPeriodSec)
    x[0] = 0
    for k in range(1, Nt):
        x[k] = x[k-1] + multiplier * np.random.normal(0.0, 1.0)
    
    fig, ax = plt.subplots(1, 1, figsize=normalFigSize)
    ax.plot(t, x, lw=1.0, color='blue')
    ax.set_xlim(t_min, t_max)
    #ax.set_ylim(-10.0, 10.0)
    ax.set_xlabel('$t$',fontsize=18)
    ax.set_ylabel('$X(t)$', fontsize=18)
    #ax.grid(showGrid)
    
    display(Audio(data=x, rate=samplingRateHz))
    return x

v = interactive(soundSamplePaths_WP, {'manual': True})
display(v)

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='$\\sigma^2$', max=2.0, read…

**Instructions:**

* To get a different sample path and audio each time, click the `Run Interact` button.
* You can download the generated audio as a `WAV` file by clicking on the three vertical dots to the left of the audio widget and select `Download`. 

**Observations:**

* A WP sounds softer than a WGN. This should not be a surprise, because one can think of the WP to be the output of an integrator, which is driven by WGN. The integrator is a LPF that cuts off very high frequencies. 

* Changing $\sigma^2$ should change the volume of the noise, but we do not observe this, since the signal is likely first normalized in terms of loudness before it is played back to us. 

## 3. Methods for Simulating Band-limited, Stationary, Zero-Mean GPs

Sampling realizations from GPs other than WGN is a bit more involved. In this section we'll discuss two methods that can be used to draw approximate sample paths of band-limited, stationary, zero-mean GPs. These two methods will be subsequently used to simulate such RPs.

### 3.1. RARP Simulation Method

The method described below, named _Random-Amplitude Random-Phase_ (RARP) method, produces approximate sample paths (realizations) band-limited, stationary, zero-mean GPs of a given PSD $S_X$. 

In reality, these are sample paths of a band-limited, stationary, zero-mean GP, whose PSD is a piece-wise constant approximation to $S_X$. In particular, the given PSD $S_X$ is approximated in the interval $\left[\omega_k-\frac{\Delta\omega}{2}, \omega_k+\frac{\Delta\omega}{2} \right]$ by $S_X(j \omega_k)$ for some spacing $\Delta\omega > 0$ rad/sec and regularly-spaced angular frequencies $\omega_k$. Obviously, the smaller $\Delta\omega$ is, the better the approximation to the intended GP.

For a stationary, zero-mean GP $X$ with PSD $S_X$, that is band-limited to $\omega_B > 0$ rad/sec (i.e., $S_X(j \omega)=0$ for $|\omega| > \omega_B$), the RARP method draws realizations from the RP $\breve{X}$, which approximates $X$ and is given as

\begin{align*}
    \breve{X}(t) = \sum_{k=0}^{N-1} \left[ A_k \cos(\omega_k t) + B_k \sin(\omega_k t) \right]
\end{align*}

where $N \triangleq \frac{\omega_B}{\Delta\omega}$. The random variables (RVs) $\left\{ A_k \right\}_{k=0}^{N-1}$ and $\left\{ B_k \right\}_{k=0}^{N-1}$ above are i.i.d. $\mathcal{N}(0, \frac{1}{\pi} S_X(j \omega_k) \Delta\omega)$ distributed and $\omega_k \triangleq k \Delta\omega$ for $k = 0, 1, \ldots, N-1$. One can show that $\breve{X}$ is indeed a GP, if the $A_k$'s and $B_k$'s are jointly Gaussianly-distributed.

It is easy to verify that $\mathbb{E} \! \left\{ \breve{X}(t) \right\} = 0$ (zero-mean). Due to the facts that $\mathbb{E}\!\left\{ A_k A_{\ell} \right\} = \frac{1}{\pi} S_X(j \omega_k) \Delta\omega \left[ k = \ell \right]$ and $\mathbb{E}\!\left\{ A_k B_{\ell} \right\} = 0$ for all $k, \ell = 0, 1, \ldots, N-1$, its auto-correlation $R_{\breve{X}}$ is given as

\begin{align*}
    R_{\breve{X}}(t + \tau, t) & \triangleq \mathbb{E}\!\left\{ \breve{X}(t + \tau) \breve{X}(t) \right\} = \frac{\Delta\omega}{\pi}  \sum_{k=0}^{N-1} S_X(j \omega_k) \left[ \cos(\omega_k t) \cos(\omega_k (t + \tau)) +  \sin(\omega_k t) \sin(\omega_k (t + \tau)) \right] = 
    \\ 
    & = \frac{\Delta\omega}{\pi}  \sum_{k=0}^{N-1} S_X(j \omega_k) \cos(\omega_k \tau)
\end{align*}

which reveals that $\breve{X}$ is WSS (since it is also mean-stationary). Based on this, its time-averaged auto-correlation $\bar{R}_{\breve{X}}$ consides with its auto-correlation function $\breve{R}_X$ and one can show that, as $\Delta\omega \to 0$,

\begin{align*}
    \bar{R}_{\breve{X}}(\tau) = \frac{1}{\pi}  \sum_{k=0}^{N-1} S_X(j \omega_k) \cos(\omega_k \tau) \Delta\omega \overset{\Delta\omega \to 0}{\longrightarrow} \frac{1}{\pi} \int_0^{\omega_B} \!\! S_X(j \omega) \cos(\omega \tau) d\omega \equiv \bar{R}_X(\tau)
\end{align*}

When graphing the sample paths of $\breve{X}$, in order to obscure their periodicity and make them resemble more like the sample path of the GP being approximated (which is not periodic), one should choose a time interval (for graphing the path), whose width is less than $\breve{X}(t)$'s period of $\frac{2 \pi}{\Delta\omega}$. Moreover, again for graphing, one should use a maximum spacing of $\frac{\pi}{\omega_B}$ (the Nyquist rate for the sample paths) between time samples. 

In [55]:
# Implementation of the RAPR simulation method for bandlimited,
# WSS, zero-mean GPs.
#
# t_min, t_max: time range in seconds for generating the sample path of the desired GP; must have t_min < t_max.
# fs_Hz>0: (time-domain) sampling rate (frequency) in Hz. Normally, should be > 2 * f_max_Hz.
# PSDfunc: a function that receives an omega array in rad/sec and returns the value of the desired GP's PSD.
# f_max_Hz>0: bandwidth of provided PSD in Hz.
def simRAPR(t_min, t_max, fs_Hz, PSDfunc, f_max_Hz):
    omega_B = 2.0 * np.pi * f_max_Hz # bandwidth in rad/sec     
    step = 1 / fs_Hz # sec
    t = np.arange(t_min, t_max, step, dtype=float) 

    # To avoid displaying more than one period of the sample path in [t_min, t_max],
    # one must choose N larger than frequency_2 * (t_max - t_min). 
    # Obviously, the larger N, the better the quality of the sample path.
    # The default value is 1000.
    N = int(max(999, f_max_Hz * (t_max - t_min))+1) 
    DeltaOmega = omega_B / N
    k = np.arange(0, N)
    omega = k * DeltaOmega    
    
    z_A = np.random.normal(0, 1, N) # sample a first set of i.i.d. standard Gaussian RVs
    z_B = np.random.normal(0, 1, N) # sample a second set of i.i.d. standard Gaussian RVs
    
    PSD = PSDfunc(omega) # sample the desired PSD
    
    A = np.sqrt(PSD * DeltaOmega / np.pi) * z_A
    B = np.sqrt(PSD * DeltaOmega / np.pi) * z_B
    
    omega_t = np.outer(t, omega)
    x = np.dot(np.cos(omega_t), A) + np.dot(np.sin(omega_t), B)
    return t, x

### 3.2. DARP Simulation Method

A second method, named _Detetministic-Amplitude Random-Phase_ (DARP) method, is an alternative to the RARP method we just saw. Once again, the desired PSD $S_X$ is approximated by a piece-wise constant PSD in the same manner that RARP uses.  

In particular, the DARP method samples realizations from the RP $\breve{X}$ given as

\begin{align*}
    & \breve{X}(t) = \sum_{k=0}^{N-1} C_k \cos(\omega_k t + \Phi_k)
\end{align*}

where $N \triangleq \frac{\omega_B}{\Delta\omega}$ and $C_k \triangleq \sqrt{ \frac{2}{\pi} S_X(j \omega_k) \Delta\omega }$. The RVs $\left\{ \Phi_k \right\}_{k=0}^{N-1}$ are i.i.d. $\mathrm{Uniform}((-\pi, \pi])$ distributed and, as usual, $\omega_k \triangleq k \Delta\omega$ for $k = 0, 1, \ldots, N-1$. 

Based on the identities 

\begin{align*}
    & \mathbb{E} \! \left\{ \cos(\theta + \Phi_k) \right\} = 0
    \\
    & \mathbb{E} \! \left\{ \cos(\theta + \Phi_k + \Phi_{\ell}) \right\} = 0
    \\
    & \mathbb{E} \! \left\{ \cos(\theta + \Phi_k - \Phi_{\ell}) \right\} = \cos(\theta) [k = \ell]
\end{align*}

for the $\Phi_k$'s as defined above and $\theta$ some other angle, it is easy to verify that $\mathbb{E} \! \left\{ \breve{X}(t) \right\} = 0$ (zero-mean) and that its auto-correlation $R_{\breve{X}}$ is given as

\begin{align*}
    R_{\breve{X}}(t + \tau, t) & = \frac{1}{\pi}  \sum_{k=0}^{N-1} S_X(j \omega_k) \cos(\omega_k \tau) \Delta\omega \overset{\Delta\omega \to 0}{\longrightarrow} \frac{1}{\pi} \int_0^{\omega_B} \!\! S_X(j \omega) \cos(\omega \tau) d\omega \equiv \bar{R}_X(\tau)
\end{align*}

which reveals that $\breve{X}$ is WSS (since it is also mean-stationary) and that it will converge to the desired auto-correlation. 

However, unlike the approximate RP $\breve{X}$ used in RARP, this RP $\breve{X}$ is not a GP for non-zero $\Delta\omega$. Each term $\breve{X}_k(t) \triangleq C_k \cos(\omega_k t + \Phi_k)$ in $\breve{X}$'s sum is independently $\mathrm{Arcsine}(-C_k, C_k)$-distributed, i.e.

\begin{align*}
    f_{\breve{X}_k(t)}(x) = \frac{1}{\pi \sqrt{C_k^2 - x^2}} [|x| < C_k]
\end{align*}

When the number of terms $N$ in $\breve{X}$'s expression approaches infinity, the Central Limit Theorem tells us that, for fixed $t$, $\breve{X}(t)$ will become a Gaussian RV and $\breve{X}$ will become a GP.    

Finally, as was the case with the RP $\breve{X}$ used in RARP, the sample paths of $\breve{X}$ used in DARP are periodic with fundamental period $\frac{2 \pi}{\Delta\omega}$, while the GP to be approximated is not. Again, to obscure this fact, when graphing sample paths of $\breve{X}$, one should choose a time interval (for graphing the path), whose width is less than $\frac{2 \pi}{\Delta\omega}$.

All in all, when one compares the two methods, DARP utilizes half the RVs that RARP uses for the same resolution $\Delta\omega$. However, RARP's approximate RP $\breve{X}$ is always a GP, while DARP's $\breve{X}$ is only asymptotically so. In practice, both methods are used for their intended purpose.

In [56]:
# Implementation of the DAPR simulation method for bandlimited,
# WSS, zero-mean GPs.
#
# t_min, t_max: time range in seconds for generating the sample path of the desired GP; must have t_min < t_max.
# fs_Hz>0: (time-domain) sampling rate (frequency) in Hz. Normally, should be > 2 * f_max_Hz.
# PSDfunc: a function that receives an omega array in rad/sec and returns the value of the desired GP's PSD.
# f_max_Hz>0: bandwidth of desired PSD in Hz.
def simDAPR(t_min, t_max, fs_Hz, PSDfunc, f_max_Hz):
    omega_B = 2.0 * np.pi * f_max_Hz # bandwidth in rad/sec     
    step = 1 / fs_Hz # sec
    t = np.arange(t_min, t_max, step, dtype=float) 

    # To avoid displaying more than one period of the sample path in [t_min, t_max],
    # one must choose N larger than frequency_2 * (t_max - t_min). 
    # Obviously, the larger N, the better the quality of the sample path.
    # The default value is 1000.
    N = int(max(999, f_max_Hz * (t_max - t_min))+1) 
    DeltaOmega = omega_B / N
    k = np.arange(0, N)
    omega = k * DeltaOmega    
    
    Phi = np.random.normal(-np.pi, np.pi, N) # sample a first set of i.i.d. standard Gaussian RVs
    
    PSD = PSDfunc(omega) # sample the desired PSD
    
    C = np.sqrt(2.0 * PSD * DeltaOmega / np.pi)
    
    omega_t_plus_phi = np.outer(t, omega) + np.tile(Phi, (len(t), 1))
    x = np.dot(np.cos(omega_t_plus_phi), C)
    return t, x

## 4. Band-Pass Filtered GWN via Ideal Filters

Let us assume that $X$ is a WGN process of intensity $\sigma^2$ and that it acts as input to an ideal, real-valued band-pass filter (BPF) with cut-off angular frequencies $\omega_2 > \omega_1 \geq 0$. The filter's impulse response is 

\begin{align*}
    & h(t) \triangleq \frac{\omega_2}{\pi} \mathrm{sinc}(\omega_2 t) - \frac{\omega_1}{\pi} \mathrm{sinc}(\omega_1 t)
\end{align*}

and has a frequency response of 

\begin{align*}
    & H(j \omega) = [\omega_1 \leq |\omega| \leq \omega_2 ]
\end{align*}

Obviously, if $\omega_1 = 0$, the filter simplifies to an ideal low-pass filter (LPF).

Then, the filter's output $Y$ will also be a zero-mean, WSS GP (as a matter of fact, $Y$ and $X$ will be jointly WSS GPs) with PSD 

\begin{align*}
    S_Y(j \omega) = \left| H(j \omega) \right\|^2 S_X(j \omega) =  \sigma^2 [\omega_1 \leq |\omega| \leq \omega_2 ]
\end{align*}

By virtue of the Wiener-Khinchin Theorem, the inverse CTFT of this PSD, yields the RP's time-averaged auto-correlation

\begin{align*}
    \bar{R}_Y(\tau) \equiv R_Y(\tau) =  \frac{\sigma^2}{\pi} \left[ \omega_2 \mathrm{sinc}(\omega_2 \tau) - \omega_1 \mathrm{sinc}(\omega_1 \tau) \right]
\end{align*}

The average power of $Y$ is given as $P_Y = \bar{R}_Y(0) = R_Y(0) = \frac{\sigma^2}{\pi} \left( \omega_2 - \omega_1 \right)$.

### 4.1 BPF WGN Autocorrelation & PSD

Further below, we graph the process' PSD and auto-correlation function.

In [57]:
# PSD of BP-filtered WGN
# sigmaSquared>0: intensity of WGN
# omega_1>=0: first cut-off angular frequency of BPF in rad/sec
# omega_2>0: second cut-off angular frequency of BPF in rad/sec; must be > omega_1
def PSD_BPF_WGN(omega, sigmaSquared, omega_1, omega_2):
    absOmega = np.abs(omega) # PSD is an even function
    return sigmaSquared * np.less_equal(omega_1, absOmega) * np.less_equal(absOmega, omega_2) 

# Auto-correlation function of BP-filtered WGN
# Inverse CTFT of PSD_BPF_WGN
# sigmaSquared>0: intensity of WGN
# omega_1>=0: first cut-off angular frequency of BPF in rad/sec
# omega_2>0: second cut-off angular frequency of BPF in rad/sec; must be > omega_1
def ACF_BPF_WGN(tau, sigmaSquared, omega_1, omega_2):
    # Note: numpy's sinc() is the normalized sinc(), so we need to divide its arguments by pi.
    absTau = np.abs(tau) # time-averaged ACF is an even function 
    return sigmaSquared * (omega_2 * np.sinc(omega_2 * absTau / np.pi) - omega_1 * np.sinc(omega_1 * absTau / np.pi) ) / np.pi


In [58]:
# Global variables/objects for this section
frequency_1_min = 0.0 # Hz
frequency_2_max = 2.0 # Hz

f1f2PassbandSliderHz = widgets.FloatRangeSlider(
    value=[frequency_1_min, frequency_2_max / 2.0],
    min=frequency_1_min,
    max=frequency_2_max,
    step=0.1,
    description='$f_1$-$f_2$ (Hz)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [59]:
# Also uses sigmaSquaredSlider
def plot_PSD_ACF_BPF_WGN(sigmaSquared=sigmaSquaredSlider, freqRangeHz=f1f2PassbandSliderHz, showGrid=True):
    
    frequency_1 = freqRangeHz[0] # Hz
    frequency_2 = freqRangeHz[1] # Hz
        
    omega_1 = 2 * np.pi * frequency_1 # rad/sec
    omega_2 = 2 * np.pi * frequency_2 # rad/sec
    
    omega_2_max = 2 * np.pi * frequency_2_max # rad/sec
    omega_min = - 1.5 * omega_2_max
    omega_max = 1.5 * omega_2_max
    Nomega = 1000
    omega = np.linspace(omega_min, omega_max, Nomega, dtype=float) 
    f = omega / (2.0 * np.pi)
    Sx = PSD_BPF_WGN(omega, sigmaSquared, omega_1, omega_2)

    tau_min = -10.0
    tau_max = 10.0
    Ntau = 1000
    tau = np.linspace(tau_min, tau_max, Ntau, dtype=float) 
    Rx = ACF_BPF_WGN(tau, sigmaSquared, omega_1, omega_2)
    max_abs_Rx = sigmaSquared_max * omega_2_max / np.pi
    
    fig, ax = plt.subplots(1, 2, figsize=largeFigSize)
    ax[0].plot(f, Sx, lw=1.0, color='blue')
    ax[0].set_xlim(omega_min / (2.0 * np.pi), omega_max / (2.0 * np.pi))
    ax[0].set_ylim(-1.25 * sigmaSquared_max, 1.25 * sigmaSquared_max)
    ax[0].set_xlabel('$f$ (Hz)',fontsize=18)
    ax[0].set_ylabel('$S_Y(j \omega)$', fontsize=18)
    ax[0].grid(showGrid)

    ax[1].plot(tau, Rx, lw=1.0, color='blue')
    ax[1].set_xlim(tau_min, tau_max)
    ax[1].set_ylim(-1.1 * max_abs_Rx, 1.1 * max_abs_Rx)
    ax[1].set_xlabel('tau (sec)',fontsize=18)
    ax[1].set_ylabel('$R_Y$(tau)', fontsize=18)
    ax[1].grid(showGrid)

    
v = interactive(plot_PSD_ACF_BPF_WGN)
display(v)

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='$\\sigma^2$', max=2.0, read…

**Instructions:**

* Use the interactive controls to see the changes in PSD and (time-averaged) ACF of the band-pass filtered WGN.
* Use the passband slider to select the cut-off frequencies $f_1$ and $f_2$ (both measured in Hz) of the BPF. When, $f_1=0$, the filter simplifies to a LPF.

**Observations:**

* As we increase the width of the passband, $R_Y(0)$ increases. This makes sense, as increasing this width, increases the average power of the GP, which equals the integral of its PSD. This coincides with $R_Y(0)$. 
* The lower the cut-off frequency $f_2$ is, the larger the lag values $\tau$, for which two values of $Y$, $Y(t + \tau)$ and $Y(t)$, may be correlated.
* For lags $\tau_o$, for which $R_Y(\tau_o) = 0$, the two GPs $Y(t + \tau_o)$ and $Y(t)$ are uncorrelated and, hence, independent.

### 4.2 BPF WGN Simulation

In the next demonstration, we will use both RARP and DARP methods to generate approximate sample paths of $Y$.

In [60]:
# Global variables & objects

# Simulation method picker
methodPicker = widgets.ToggleButtons(
    options=['RARP', 'DARP'],
    value = 'RARP',
    description='Method:',
    disabled=False,
    button_style='',
    tooltips=['Description of slow', 'Description of regular'],
)

In [61]:
# Also uses sigmaSquaredSlider and f1f2PassbandSliderHz
def plotSamplePaths_BPF_WGN(sigmaSquared=sigmaSquaredSlider, freqRangeHz=f1f2PassbandSliderHz, method=methodPicker, showGrid=True):
    
    # Cut-off frequencies
    frequency_1 = freqRangeHz[0] # Hz
    frequency_2 = freqRangeHz[1] # Hz
    fs = 2.0 * frequency_2_max # Nyquist rate for fastest signal
      
    t_min = 0.0
    t_max = 100.0
    myPSDfunc = functools.partial(PSD_BPF_WGN, sigmaSquared=sigmaSquared, omega_1 = 2 * np.pi * frequency_1, 
                                  omega_2 = 2 * np.pi * frequency_2)

    if method == 'RARP':
        t, x = simRAPR(t_min, t_max, fs, myPSDfunc, frequency_2_max)
    elif method == 'DARP':
        t, x = simDAPR(t_min, t_max, fs, myPSDfunc, frequency_2_max)

    
    fig, ax = plt.subplots(1, 1, figsize=normalFigSize)
    ax.plot(t, x, lw=1.0, color='blue')
    ax.set_xlim(t_min, t_max)
    ax.set_ylim(-4.0, 4.0)
    ax.set_xlabel('$t$ (sec)',fontsize=18)
    ax.set_ylabel('$y(t)$', fontsize=18)
    ax.grid(showGrid)
    
v = interactive(plotSamplePaths_BPF_WGN,  {'manual': True})
display(v)

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='$\\sigma^2$', max=2.0, read…

**Instructions:**

* To get an approximate sample path each time, click the `Run Interact` button.
* Use the passband slider to select the cut-off frequencies $f_1$ and $f_2$ (both measured in Hz) of the BPF filter. When, $f_1=0$, the filter simplifies to a LPF.
* Select between the `RARP` and `DARP` methods to sample a realization.

**Observations:**

* The time average of the sample paths is approximately $0$ (as expected).
* Increasing the intensity $\sigma^2$ increaces the magnitude of the sample paths' fluctuations.
* For $f_1=0$ and $f_2=0.2$, the noise we are obtaining is narrow baseband. Its sample paths reflect that as they look like sums of a few slow-varying sinusoids.
* For $f_1=0$ and as we increase $f_2$ to 2, the sample paths become "choppier" with wilder magintude fluctuations, as more noise frequencies are allowed to pass through the LPF.
* For $f_1=0.9$ and $f_2=1.1$, i.e., when our filter is a proper BPF, the sample paths indeed look like a slow-varying signal modulating a higher-frequncy sinusoid.  

### 4.3 BPF WGN Audio Sample

The next demonstration produces an audio of the type of noises investigated in the previous demonstration.

In [62]:
# Global variables/objects
frequency1KHz_min = 0.0 # KHz
frequency2KHz_max = 0.5 # KHz 

f1f2PassbandSliderKHz = widgets.FloatRangeSlider(
    value=[frequency1KHz_min, frequency2KHz_max],
    min=frequency1KHz_min,
    max=frequency2KHz_max,
    step=0.01,
    description='$f_1$-$f_2$ (KHz)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.2f',
)

In [63]:
# Also uses durationSeconds declared earlier.
def soundSamplePaths_BPF_WGN(freqRangeKHz = f1f2PassbandSliderKHz):    
    f_2_max = 1000.0 * frequency2KHz_max # Hz
            
    omega_1 = 2 * np.pi * 1000.0 * freqRangeKHz[0] # rad/sec
    omega_2 = 2 * np.pi * 1000.0 * freqRangeKHz[1] # rad/sec
            
    t_min = 0.0
    t_max = durationSeconds
    # Note to self: for some reason, signals that are sampled at less than 3KHz,
    # will not play in the widget.
    fs = 6 * 1000 * frequency2KHz_max # Hz; sampling rate much above the Nyquist rate of the signal
    myPSDfunc = functools.partial(PSD_BPF_WGN, sigmaSquared=1.0, omega_1 = omega_1, 
                                  omega_2 = omega_2)
    t, x = simRAPR(t_min, t_max, fs, myPSDfunc, f_2_max)
        
    fig, ax = plt.subplots(1, 1, figsize=normalFigSize)
    ax.plot(t, x, lw=1.0, color='blue')
    ax.set_xlim(t_min, t_max)
    #ax.set_ylim(-10.0, 10.0)
    ax.set_xlabel('$t$',fontsize=18)
    ax.set_ylabel('$X(t)$', fontsize=18)
    #ax.grid(showGrid)
    
    display(Audio(data=x, rate=fs))
    return x

v = interactive(soundSamplePaths_BPF_WGN, {'manual': True})
display(v)

interactive(children=(FloatRangeSlider(value=(0.0, 0.5), continuous_update=False, description='$f_1$-$f_2$ (KH…

**Instructions:**

* To get an approximate sample path and audio each time, click the `Run Interact` button.
* Use the passband slider to select the cut-off frequencies $f_1$ and $f_2$ (both measured in KHz) of the BPF filter. When, $f_1=0$, the filter simplifies to a LPF.
* You can download the generated audio as a `WAV` file by clicking on the three vertical dots to the left of the audio widget and select `Download`. 

**Observations:**

* By choosing a wide passband, one can get a variety of "outer-space" or "underwater" (muffled) noise sounds.
* As a matter of fact, choosing $f_1=0$ and $f_2=0.5$ KHz (the widest passband of this demonstration), the noise sounds similar to the underwater noise present in the [Bloop sound](https://en.wikipedia.org/wiki/File:Bloop_real.ogg) (here, sped up 16 times), although Bloop's backrground noise seems to be low-pass filtered at a higher cut-off frequency. Hence, at first impression, one could say that underwater noise picked up by hydrophones resembles low-pass filtered WGN. More on the Bloop signal can be found in the references below.

**References:**

* [The Bloop (Wikipedia)](https://en.wikipedia.org/wiki/Bloop)
* [The Bloop Heard Beneath The Sea | Sound Mysteries (Gizmodo on Youtube)](https://www.youtube.com/watch?v=3BWO3D8x3AQ)

## 5. Bandlimited Baseband Colored Gaussian Noise

Any zero-mean, stationary GP with a non-flat spectrum (PSD) is refered to as a _Colored Gaussian Noise_ (CGN). 

Let's consider a stationary GP $X$ with the following PSD:   

\begin{align*}
    & S_X(j \omega) \triangleq \pi e^{-a |\omega|} \ [|\omega| \leq \omega_B] & a > 0, \omega_B > 0
\end{align*}

We say that $X$ has a _truncated/bandlimited Laplacian_ spectrum. Using the inverse CTFT, we obtain its time-averaged auto-correlation $\bar{R}_X$ and, hence, its auto-correlation $R_X$ (since $X$ is WSS, these two functions coincide) as

\begin{align*}
    & R_X(\tau) = \frac{1}{\tau^2 + a^2} \left[ a + e^{-a \omega_B} \left(\tau \sin(\omega_B \tau) - a \cos(\omega_B \tau) \right) \right]
\end{align*}

Since $S_X$ does not contain any impulse at $\omega = 0$, or, equivalently, since $\lim\limits_{|\tau| \to +\infty} R_X(\tau) = 0$, $X$ is a zero-mean RP. Hence, $X$ is a CGN with average power $P_X = \bar{R}_X(0) = R_X(0) = \frac{1}{a} (1 - e^{-a \omega_B})$.


### 5.1 Bandlimited Baseband CGN Autocorrelation & PSD

Let's see how its PSD and auto-corerlation functions look like.

In [64]:
# Badnlimited double-exponential PSD of CGN
# alpha>0: exponent parameter used in the PSD 
# omega_B>0: cut-off angular frequency of PSD.
def PSD_Bandlimited_CGN(omega, alpha, omega_B):
    return np.pi * np.exp(- alpha * np.abs(omega)) * np.less_equal(np.abs(omega), omega_B).astype(float)

# Auto-correlation function of the related CGN.
# Inverse CTFT of PSD_Bandlimited_CGN
# alpha>0: exponent parameter used in the PSD 
# omega_B>0: cut-off angular frequency of PSD.
def ACF_Bandlimited_CGN(tau, alpha, omega_B):
    tmp = alpha + np.exp(- alpha * omega_B) * ( tau * np.sin(omega_B * tau) - alpha * np.cos(omega_B * tau))
    return tmp / (tau**2 + alpha**2)

In [65]:
# Global variables & objects
log10a_min = -5.0
log10a_max = 1.0

log10aSlider = widgets.FloatLogSlider(
    value=0.1,
    base=10,
    min=log10a_min, # min exponent of base
    max=log10a_max, # max exponent of base
    step=0.5, # exponent step
    description='$a$',
#    readout_format='.6f',
)

fB_Hz_min = 0.1 # Hz
fB_Hz_max = 2.0 # Hz

fBSliderHz = widgets.FloatSlider(
    value=fB_Hz_max,
    min=fB_Hz_min,
    max=fB_Hz_max,
    step=0.1,
    description='$f_B$ (Hz)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [66]:
def plot_PSD_ACF_Bandlimited_CGN(a=log10aSlider, fB_Hz=fBSliderHz, showGrid=True):
    
    omega_B = 2 * np.pi * fB_Hz # rad/sec
    omega_B_max = 2 * np.pi * fB_Hz_max # rad/sec 
      
    omega_min = - 1.5 * omega_B_max
    omega_max = 1.5 * omega_B_max
    Nomega = 1000
    omega = np.linspace(omega_min, omega_max, Nomega, dtype=float) 
    f = omega / (2.0 * np.pi)
    Sx = PSD_Bandlimited_CGN(omega, a, omega_B)
    PSD_max = np.pi

    tau_min = -10.0
    tau_max = 10.0
    Ntau = 1000
    tau = np.linspace(tau_min, tau_max, Ntau, dtype=float) 
    Rx = ACF_Bandlimited_CGN(tau, a, omega_B)
    
    fig, ax = plt.subplots(1, 2, figsize=largeFigSize)
    ax[0].plot(f, Sx, lw=1.0, color='blue')
    ax[0].set_xlim(omega_min / (2.0 * np.pi), omega_max / (2.0 * np.pi))
    ax[0].set_ylim(-0.1 * PSD_max, 1.1 * PSD_max)
    ax[0].set_xlabel('$f$ (Hz)',fontsize=18)
    ax[0].set_ylabel('$S_X(j \omega)$', fontsize=18)
    ax[0].grid(showGrid)

    ax[1].plot(tau, Rx, lw=1.0, color='blue')
    ax[1].set_xlim(tau_min, tau_max)
    ax[1].set_ylim(-4.0, 14.0)
    ax[1].set_xlabel('tau (sec)',fontsize=18)
    ax[1].set_ylabel('$R_X$(tau)', fontsize=18)
    ax[1].grid(showGrid)

    
v = interactive(plot_PSD_ACF_Bandlimited_CGN)#, {'manual': True})
display(v)

interactive(children=(FloatLogSlider(value=0.1, description='$a$', max=1.0, min=-5.0, step=0.5), FloatSlider(v…

**Instructions:**

* Use the interactive controls to see the changes in PSD and (time-averaged) ACF of the bandlimited CGN.

**Observations:**

* Keeping fixed $f_B = 2$ Hz, as we decrease the value of $a$, the PSD $S_X$ becomes flatter in its passband and, hence, increasingly resembles the spectrum of low-pass filtered WGN. Hence, its auto-correlation resembles a $\mathrm{sinc}(\cdot)$ function.
* On the other hand, as we increase $a$, its value affects the PSD's effective bandwidth, which becomes smaller. This results in a flatter auto-correlation function.

### 5.2 Bandlimited Baseband CGN Simulation

Next, we'll draw realizations from this RP using the RARP method (instead of DARP, for no particular reason).

In [67]:
# Also uses log10aSliderHz, fBSliderHz and fB_Hz_max
def plotSamplePaths_Bandlimited_CGN(a=log10aSlider, fB_Hz=fBSliderHz, showGrid=True):
    
    fs = 2.0 * fB_Hz_max # Nyquist rate for fastest signal
    
    t_min = 0.0
    t_max = 100.0
    myPSDfunc = functools.partial(PSD_Bandlimited_CGN, alpha = a, omega_B = 2 * np.pi * fB_Hz)

    t, x = simRAPR(t_min, t_max, fs, myPSDfunc, fB_Hz_max) #frequency_2_max)
    
    fig, ax = plt.subplots(1, 1, figsize=normalFigSize)
    ax.plot(t, x, lw=1.0, color='blue')
    ax.set_xlim(t_min, t_max)
    ax.set_ylim(-10.0, 10.0)
    ax.set_xlabel('$t$ (sec)',fontsize=18)
    ax.set_ylabel('$x(t)$', fontsize=18)
    ax.grid(showGrid)
    
v = interactive(plotSamplePaths_Bandlimited_CGN,  {'manual': True})
display(v)

interactive(children=(FloatLogSlider(value=0.1, description='$a$', max=1.0, min=-5.0, step=0.5), FloatSlider(v…

**Instructions:**

* To get an approximate sample path each time, click the `Run Interact` button.
* Use the frequency slider to select the cut-off frequency $f_B$ (measured in Hz) of the LPF filter. 

**Observations:**

* For fixef $f_B = 2$ Hz, when $a$ is less than $0.1$ the process seems to maintain a certain average amplitude. Starting, though, with $a = 0.1$ and increasing it beyond this value decreases the realizations' amplitude.
* For a value of $a=10$, the sample paths' appearance remains unaffected from changes in $f_B$. In essense, the large value of $a$ on its own makes the bandwidth of the process very narrow. We saw this phenomenon in the previous section, when inspecting the PSD of processes with such characteristics.

### 5.3 Bandlimited Baseband CGN Audio Sample

Let's listen now how such processes sound.

In [68]:
# Global variables & objects
log10a_min = -4.0
log10a_max = -2.0

log10aSlider2 = widgets.FloatLogSlider(
    value=log10a_max,
    base=10,
    min=log10a_min, # min exponent of base
    max=log10a_max, # max exponent of base
    step=0.1, # exponent step
    description='$a$',
#    readout_format='.6f',
)


fB_KHz_min = 0.1 # KHz
fB_KHz_max = 2.0 # KHz

fBSliderKHz = widgets.FloatSlider(
    value=fB_KHz_max,
    min=fB_KHz_min,
    max=fB_KHz_max,
    step=0.1,
    description='$f_B$ (KHz)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

In [69]:
# Also uses durationSeconds and log10aSlider declared earlier.
def soundSamplePaths_Bandlimited_CGN(a=log10aSlider2, fB_KHz=fBSliderKHz):    

    myPSDfunc = functools.partial(PSD_Bandlimited_CGN, alpha = a, omega_B = 2 * np.pi * 1000.0 * fB_KHz)

    t_min = 0.0
    t_max = durationSeconds
    fs = 4 * 1000 * fB_KHz_max # Hz; sampling rate much above the Nyquist rate of the signal
    t, x = simRAPR(t_min, t_max, fs, myPSDfunc, 1000 * fB_KHz_max)
        
    fig, ax = plt.subplots(1, 1, figsize=normalFigSize)
    ax.plot(t, x, lw=1.0, color='blue')
    ax.set_xlim(t_min, t_max)
    #ax.set_ylim(-10.0, 10.0)
    ax.set_xlabel('$t$',fontsize=18)
    ax.set_ylabel('$X(t)$', fontsize=18)
    #ax.grid(showGrid)
    
    display(Audio(data=x, rate=fs))
    return x

v = interactive(soundSamplePaths_Bandlimited_CGN, {'manual': True})
display(v)

interactive(children=(FloatLogSlider(value=0.0001, description='$a$', max=-2.0, min=-4.0), FloatSlider(value=2…

**Instructions:**

* To get an approximate sample path and audio each time, click the `Run Interact` button.
* Use the passband slider to select the cut-off frequency $f_B$ (measured in KHz) of the LPF filter.
* You can download the generated audio as a `WAV` file by clicking on the three vertical dots to the left of the audio widget and select `Download`. 

**Observations:**

* For high values of $a$, we get a "rumbling" noise.
* On the other hand, when $a$ is very small, the noise sounds like low-pass filtered WGN that we have encountered before. This is not a surprise, since, in this case, the PSD of the CGN has an almost flat passband and resembles that of low-pass filtered WGN. 